In [24]:
from CwnGraph import CwnBase, CwnSense

In [3]:
import igraph as ig

In [35]:
from tqdm.autonotebook import tqdm

/Users/seantyh/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [6]:
cwn = CwnBase()

In [8]:
all_chars = cwn.find_lemma("^.$")

In [13]:
char_nodes = set(x.lemma for x in all_chars)

In [19]:
from itertools import chain
sense_nodes = list(set(chain.from_iterable(x.senses for x in all_chars)))

In [55]:
sense_nodes[0].relations[0]

('is_synset', <CwnSynset[syn_014828]: 治理杭州的地方政府。>, 'forward')

In [56]:
sense_relations = [(x.id, r[1].id, r[0]) for x in sense_nodes for r in x.relations if type(r[1]) is CwnSense and r[2]=="forward"]

In [57]:
len(sense_relations)

30664

In [59]:
G = ig.Graph(directed=False)
for x in tqdm(char_nodes):
    G.add_vertex(x)

for s in tqdm(sense_nodes):
    G.add_vertex(s.id)
    try:
        G.add_edge(s.lemmas[0].lemma, s.id)
    except Exception as ex:
        print(ex)
        break


In [60]:
vs = set(x["name"] for x in G.vs)
for r in tqdm(sense_relations):
    src_id, tgt_id, rel_type = r
    if tgt_id in vs:
        G.add_edge(src_id, tgt_id, relation=rel_type)

In [62]:
G.summary()

'IGRAPH UN-- 21713 34331 -- \n+ attr: name (v), relation (e)'

In [68]:
[G.vs[x] for x in G.get_shortest_paths("重", "覆")[0]]

[igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 345, {'name': '重'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 15555, {'name': '05207612'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 20937, {'name': '06593813'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 4290, {'name': '緊'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 16214, {'name': '06593809'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 20291, {'name': '06508307'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 4611, {'name': '密'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 6577, {'name': '06508305'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 14595, {'name': '05235009'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 2838, {'name': '壞'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 21046, {'name': '05235008'}),
 igraph.Vertex(<igraph.Graph object at 0x7faf10a7a250>, 21185, {'name': '06709316'}),
 i

In [69]:
from mesh_import import mesh

In [91]:
import pandas as pd
from opencc import OpenCC
import numpy as np
cc = OpenCC('s2t')

In [74]:
m2 = pd.read_csv(mesh.get_data_dir() / "cns/m2_cld.csv", index_col=0)

In [79]:
tw_words = [cc.convert(x) for x in m2["word"]]

In [96]:
dists = []
for x in tqdm(tw_words):
    try:
        dists.append(len(G.get_shortest_paths(x[0], x[1])[0]))
    except:
        dists.append(np.nan)

/Users/seantyh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Couldn't reach some vertices at /Users/travis/build/igraph/python-igraph/vendor/source/igraph/src/structural_properties.c:768
  after removing the cwd from sys.path.


In [98]:
m2["sensePathDist"] = dists

In [100]:
m2.to_csv(mesh.get_data_dir() / "cns/m2_cld_sensePath.csv")